In [4]:
import credentials # Api key is stored in this file, remove to avoid errors if you clone from github

import pvdeg
import pvlib
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy.linalg import cholesky
from scipy import stats

First get weather data and metadata for a desired location (latitude and logitude)

In [2]:
# change to desired values (currently Miami)
latitude = 25.783388
longitude = -80.189029

API_KEY = credentials.API_KEY # my personal NREL api key
email ='tobin.ford@nrel.gov' # replace these values with your appropriate information and remove and comment out first line of first block (import credentials)

# reads NSRDB data 
weather_df, meta = pvlib.iotools.get_psm3(latitude, longitude, API_KEY, email, names='2019', map_variables=True)

c:\ProgramData\anaconda3\envs\monte-carlo\lib\site-packages\pvlib\iotools\psm3.py:183: pvlibDeprecationWarning: The ``get_psm3`` function will default to leap_day=True starting in pvlib 0.11.0. Specify leap_day=True to enable this behavior now, or specify leap_day=False to hide this warning.
  warnings.warn(


User has 3 parameters for initial implementation: See Kempe's "Deg Miami" tab in excel<br>



activation energy, Ea <br>
irradiance relation, x<br>
ln(R0)<br>

|           |   Ea   |   x   | ln(R0) |
|:---------:|:-----:|:----:|:-------:|
|   **Ea**  |   1   |   a  |   b     |
|   **x**   |   a   |   1  |   c     |
| **ln(R0)**|   b   |   c  |   1     |

Notice symmetry across diagonal <br>

In [102]:
# USER ENTERED VALUES
# Correlation Coefficients
Ea_X = 0.0269
Ea_lnR0 = -0.9995 
X_lnR0 = -0.0400

# Activation Energy
mean_Ea = 62.08 # average
sd_Ea = 7.3858 # standard deviation

# Irradiance relation
mean_X = 0.0341 # average
sd_X = 0.0992757 # standard deviation

# ln(R0)
mean_lnR0 = 13.7223084 
sd_lnR0 = 2.47334772

# number of iterations
n = 20000


In [118]:
# notice symmetry of matrix
A = np.array([[1,   Ea_X,   Ea_lnR0],
              [Ea_X,    1,   X_lnR0],
              [Ea_lnR0, X_lnR0,   1]])

# conceptually similar to the square root of a matrix
A_decomp = cholesky(A, lower=True) 

# generating standard distribution of points for each
#ea = np.random.normal(mean_Ea, sd_Ea, 20_000)
#x, lnR0 = np.random.normal(mean_Ea, sd_Ea, 20_000), np.random.normal(mean_X, sd_X, 20_000), np.random.normal(mean_lnR0, sd_lnR0, 20_000)


# an attempt to try what I am seeing in the excel macro data
# only ea has standard distribution based on mean and sd
# x and lnR0 have the regular standard distribution to be used in decomposition?
# this mimics the behavior in the macro

#ea = np.random.normal(mean_Ea, sd_Ea, n) 

ea = np.random.normal(loc=0, scale=1, size=n)
x = np.random.normal(loc=0, scale=1, size=n)
lnR0 = np.random.normal(loc=0, scale=1, size=n)


# transpose built in so I don't have to do it with another function
samples_matrix = np.array([ea, x, lnR0])

data = {
    'ea': ea,
    'x': x,
    'lnR0': lnR0
}

uncorrelated_df = pd.DataFrame(data)


In [105]:
A

array([[ 1.    ,  0.0269, -0.9995],
       [ 0.0269,  1.    , -0.04  ],
       [-0.9995, -0.04  ,  1.    ]])

In [106]:
A_decomp

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.0269    ,  0.99963813,  0.        ],
       [-0.9995    , -0.0131182 ,  0.02876913]])

In [68]:
# these are one trial of kempe's before he does the decomp
# Ea: 63.51
# X: 0.7078
# LnR0; -0.324

# now my data matches his pre-decomp, changed distributions (see above block + comments)

In [69]:
print(ea.min())
print(ea.max())
print()

print(x.min())
print(x.max())
print()

print(lnR0.min())
print(lnR0.max())
print()

27.07733841747733
91.4510164750473

-0.3391959505357475
0.4322401261723464

3.526823005435915
24.161556618039064



In [107]:
# correlated stats pre-input to function

# not entirely sure what I should do here,
# do i multiply the transpose by the lower cholesky to get correlated input data for the equation on the spreadsheet
# using MonteCarloEaLnRoX

# this gives us correlated input samples
# looks like what kempe's code was doing in "MonteCarloEaLnRoX" but it is hard to tell what's happening in his code
correlated_samples = np.matmul(A_decomp, samples_matrix)

correlated_df = pd.DataFrame(correlated_samples.T, columns=['ea', 'x', 'lnR0'])

sol_pos = pvdeg.spectral.solar_position(weather_df, meta)
poa_irradiance = pvdeg.spectral.poa_irradiance(weather_df, meta)
temp_mod = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_irradiance, conf='open_rack_glass_polymer')
# we only care about global irradiance in this case


In [71]:
print(correlated_df)

              ea         x       lnR0
0      64.988895 -0.017102  15.440178
1      64.470212  0.087281  16.249761
2      59.839005  0.185417  16.050443
3      49.942103  0.061757  15.015683
4      55.203570 -0.035814  14.708526
...          ...       ...        ...
19995  57.456498 -0.007369  12.506731
19996  63.807519  0.001583   8.984753
19997  57.505258  0.046737  14.424287
19998  65.870998  0.101847   9.487176
19999  52.036715 -0.115932  14.741607

[20000 rows x 3 columns]


In [108]:
# attempting to compare my function input values to kempe's from macro
# Ea min:30.187
# Ea max:89.94

# lnR0 min: 4.477
# lnR0 max: 24.31

# X min:-0.387
# X max: 0.415

# uncorrelated samples of mine, random but should be sim
'''
ea
34.50720101130711
95.14753086962446

lnR0
4.258833206368452
23.659585717113735

x
-0.349302212696885
0.4300323062342357
'''
# these are very similar to kempes that he was feeding to the arrhenius equation in the macro, not similar to my correlated variables at all?
# what happens if i feed my uncorrelated variables to the function


print(correlated_df['ea'].mean())
print(correlated_df['ea'].std())

print(correlated_df['lnR0'].mean())
print(correlated_df['lnR0'].std())

print(correlated_df['x'].mean())
print(correlated_df['x'].std())

# can conclude there is something weird happening with my input data. 

-0.010298182357210896
0.9981098303041948
0.010800181449826758
0.9979808202407752
0.004504390091954463
0.9956684416383704


In [113]:
Ea_New = sd_Ea * ea.std() + mean_Ea
lnR0_New = sd_lnR0 * lnR0.std() + mean_lnR0
x_NEW = sd_X * x.std() + mean_X

In [114]:
print(Ea_New)
print(lnR0_New)
print(x_NEW)

69.45165528636734
13.7223084
0.0341


In [119]:
uncorrelated_df.head(5)

,ea,x,lnR0
0,2.059591,0.761724,-0.750288
1,-1.715056,-0.476199,0.303484
2,0.420392,-1.344562,1.134772
3,1.499597,-0.796474,0.174099
4,1.113856,-0.801054,0.066175


In [180]:
temp = np.matrix(np.matmul(A_decomp, samples_matrix))
print(temp.shape)

sd_mat = np.matrix([sd_Ea, sd_X, sd_lnR0]) 
print(sd_mat.shape)

print(temp, end='\n')
print(sd_mat)


(3, 20000)
(1, 3)
[[ 2.05959133 -1.71505622  0.42039215 ...  0.44976816 -0.99888557
  -2.09302951]
 [ 0.81685117 -0.52216167 -1.33276686 ...  0.66983073 -0.15962617
   1.24176431]
 [-2.09013911  1.72917653 -0.36989731 ... -0.44474703  1.00233782
   2.06899747]]
[[7.3858     0.0992757  2.47334772]]


In [188]:
sd_mat_transpose = np.transpose(sd_mat)

result = np.multiply(temp, sd_mat_transpose) + np.transpose(np.matrix([mean_Ea, mean_X, mean_lnR0]))
print(result)


[[ 7.72917296e+01  4.94129378e+01  6.51849324e+01 ...  6.54018977e+01
   5.47024310e+01  4.66213027e+01]
 [ 1.15193471e-01 -1.77379657e-02 -9.82113633e-02 ...  1.00597915e-01
   1.82529998e-02  1.57377021e-01]
 [ 8.55266759e+00  1.79991632e+01  1.28074237e+01 ...  1.26222944e+01
   1.62014384e+01  1.88396586e+01]]


In [190]:
print(result[0].mean())
print(result[0].std())
print()
print(result[1].mean())
print(result[1].std())
print()
print(result[2].mean())
print(result[2].std())

62.07521558072824
7.4422903843661

0.03412382762871065
0.0991614206324547

13.72347143507861
2.4917183935255873


In [194]:
print("EA_X:", np.corrcoef(result[0], result[1]))
print("Ea_lnR0:", np.corrcoef(result[0], result[2]))
print("X_lnR0:", np.corrcoef(result[1], result[2]))

EA_X: [[1.         0.02573158]
 [0.02573158 1.        ]]
Ea_lnR0: [[ 1.        -0.9995067]
 [-0.9995067  1.       ]]
X_lnR0: [[ 1.        -0.0389422]
 [-0.0389422  1.       ]]


In [204]:
correlated_df = pd.DataFrame(np.transpose(result), columns=['ea', 'x', 'lnR0'])

In [205]:
correlated_df.head()

,ea,x,lnR0
0,77.291730,0.115193,8.552668
1,49.412938,-0.017738,17.999163
2,65.184932,-0.098211,12.807424
3,73.155725,-0.040937,10.053368
4,70.306718,-0.042422,10.999432


In [211]:
def mikeArrhenius(weather_df, samples):

    #['e', 'x', 'lnR0']
    
    d = [(np.exp(z) * np.exp((x / 8.31446261815324E-03) / (273.15 + temp_mod)) * (poa_irradiance['poa_global'] / 1000 ) ** y) for x, y, z in zip(samples['ea'], samples['x'], samples['lnR0'])]
    # -> suns of irradiance

    

    return (pd.DataFrame(d) / len(weather_df))

In [212]:
temp2 = mikeArrhenius(weather_df=weather_df, samples=correlated_df) 

# trying with uncorrelated variables

In [213]:
temp2.head(24) 
# each row shows the degredation rate for that given monte carlo trial iteration
# data frame indexes are integers corresponding to each trial

,0
0,3.220866e+16
1,inf
2,inf
3,inf
4,inf
5,inf
6,inf
7,1.239178e+16
8,2.248793e+16
9,inf
